# Libraries Loading

In [1]:
import utils.dataHandlingUtils as dU

In [2]:
import os
import torchaudio
import torch

# Data Preprocessing

In [ ]:
audio_file = "your_audio_file.wav" 
folder_path = "your_folder_path"

waveform, sample_rate = dU.load(audio_file)
dU.plot_spectrogram(waveform, sample_rate)
dU.play_audio(waveform, sample_rate)

print("Sample Rate:", dU.get_sample_rate(audio_file))
print("Files in folder:", dU.count_files_in_folder(folder_path))

is_audio, common_sample_rate = dU.check_sample_rate_consistency(folder_path)
if is_audio:
    print("All audio files have the same sample rate:", common_sample_rate)
else:
    print("Audio files have different sample rates.")
